In [ ]:
import numpy as np
import panel as pn

pn.extension()

One of the main design goals for Panel was that it should make it possible to seamlessly transition back and forth between interactively prototyping a dashboard in the notebook or on the commandline to deploying it as a standalone server app. This section shows how to display panels interactively, embed static output, save a snapshot, and deploy as a separate web-server app. For more information about deploying Panel apps to various cloud providers see the [Server Deployment](Server_Deployment.ipynb) documentation.

## Configuring output

As you may have noticed, almost all the Panel documentation is written using notebooks. Panel objects display themselves automatically in a notebook and take advantage of Jupyter Comms to support communication between the rendered app and the Jupyter kernel that backs it on the Python end. To display a Panel object in the notebook is as simple as putting it on the end of a cell. Note, however, that the ``panel.extension`` first has to be loaded to initialize the required JavaScript in the notebook context. In recent versions of JupyterLab this works out of the box but for older versions (`<3.0`) the PyViz labextension has to be installed with:

    jupyter labextension install @pyviz/jupyterlab_pyviz

### Optional dependencies

Also remember that in order to use certain components such as Vega, LaTeX, and Plotly plots in a notebook, the models must be loaded using the extension. If you forget to load the extension, you should get a warning reminding you to do it. To load certain JS components, simply list them as part of the call to ``pn.extension``:

    pn.extension('vega', 'katex')

Here we've ensured that the Vega and LaTeX JS dependencies will be loaded.

### Initializing JS and CSS 

Additionally, any external ``css_files``, ``js_files`` and ``raw_css`` needed should be declared in the extension. The ``js_files`` should be declared as a dictionary mapping from the exported JS module name to the URL containing the JS components, while the ``css_files`` can be defined as a list:

    pn.extension(js_files={'deck': https://unpkg.com/deck.gl@~5.2.0/deckgl.min.js},
                 css_files=['https://api.tiles.mapbox.com/mapbox-gl-js/v0.44.1/mapbox-gl.css'])

The ``raw_css`` argument allows defining a list of strings containing CSS to publish as part of the notebook and app.

Providing keyword arguments via the ``extension`` is the same as setting them on ``pn.config``, which is the preferred approach outside the notebook. ``js_files`` and ``css_files`` may be set to your chosen values as follows:

    pn.config.js_files  = {'deck': 'https://unpkg.com/deck.gl@~5.2.0/deckgl.min.js'}
    pn.config.css_files = ['https://api.tiles.mapbox.com/mapbox-gl-js/v0.44.1/mapbox-gl.css']

## Display in the notebook

#### The repr
    
Once the extension is loaded, Panel objects will display themselves if placed at the end of cell in the notebook:

In [ ]:
pane = pn.panel('<marquee>Here is some custom HTML</marquee>')

pane

To instead see a textual representation of the component, you can use the ``pprint`` method on any Panel object:

In [ ]:
pane.pprint()

#### The ``display`` function

To avoid having to put a Panel on the last line of a notebook cell, e.g. to display it from inside a function call, you can use the IPython built-in ``display`` function:

In [ ]:
def display_marquee(text):
    display(pn.panel('<marquee>{text}</marquee>'.format(text=text)))
    
display_marquee('This Panel was displayed from within a function')

#### Inline apps

Lastly it is also possible to display a Panel object as a Bokeh server app inside the notebook.  To do so call the ``.app`` method on the Panel object and provide the URL of your notebook server:

In [ ]:
pane.app('localhost:8888')

The app will now run on a Bokeh server instance separate from the Jupyter notebook kernel, allowing you to quickly test that all the functionality of your app works both in a notebook and in a server context.

### ipywidgets

If the `jupyter_bokeh` package is installed it is also possible to render Panel objects as an ipywidget rather than using Bokeh's internal communication mechanisms.  You can enable ipywidgets support  globally using:

```python
pn.extension(comms='ipywidgets')
# or
pn.config.comms = 'ipywidgets'
```

This global setting can be useful when trying to serve an entire notebook using [Voilà](https://github.com/voila-dashboards/voila). Alternatively, we can convert individual objects to an ipywidget one at a time using the `pn.ipywidget()` function:

```python
ipywidget = pn.ipywidget(pane)
ipywidget
```

This approach also allows combining a Panel object with any other Jupyter-widget--based model:

```python
from ipywidgets import Accordion
Accordion(children=[pn.ipywidget(pane)])
```

To use Panel's ipywidgets support in JupyterLab, the following extensions have to be installed:
    
```
jupyter labextension install @jupyter-widgets/jupyterlab-manager
jupyter labextension install @bokeh/jupyter_bokeh
```

Additionally the `jupyter_bokeh` package should be installed using either pip:

```
pip install jupyter_bokeh
```

or using conda:

```
conda install -c bokeh jupyter_bokeh
```